## Genetic-810199576 

In [1]:
import random
import numpy
import pandas as pd 
from copy import copy, deepcopy
POPULATION_SIZE = 100
# SAMPLE_PATH = 'sample.csv'
CROSSOVER_PROBABILITY = 0.9
MUTATION_PROBABILITY = 0.1
ALPHA = 10
ELITE = 20


حال کلاس کروموزوم را تعریف میکنیم:

In [2]:
class Chromosome:
    df: pd.core.frame.DataFrame
    __fitness: float
        
    def __init__(self, lavrage: numpy.ndarray) -> None:
        self.__fitness = 0
        self.__lavrage = pd.DataFrame(lavrage, columns = ['lavrage'])
        self.__lavrage['ticker'] = self.df['ticker']
        self.update()
        
    def __lt__(self, other: 'Chromosome') -> bool:
        return self.fitness < other.fitness
    
    def update(self) -> None:
        self.__lavrage['lavrage']/=self.__lavrage['lavrage'].sum()
        self.__lavrage['return'] = self.df['return']*self.__lavrage['lavrage']
        self.__lavrage['risk'] = self.df['risk']*self.__lavrage['lavrage']
        
    @property
    def fitness(self)-> float:
        self.update()
        riskSum  = self.__lavrage['risk'].sum()
        returnSum  = self.__lavrage['return'].sum()
        self.__fitness = returnSum - ALPHA*riskSum
        if riskSum > 0.6 : self.__fitness -= 2
        return self.__fitness
    
    
    @property
    def lavrage(self) -> numpy.ndarray:
        self.update()
        return self.__lavrage['lavrage'].to_numpy()
    
    @property
    def risk(self) -> float:
        self.update()
        return self.__lavrage['risk'].sum()*100
    
    @property
    def _return(self) -> float:
        self.update()
        return self.__lavrage['return'].sum()*100
    
    def mutate(self) -> None:
        self.update()
        for i in range(len(self.__lavrage)):
            self.__lavrage.at[i, 'lavrage'] = random.uniform(0, 1)
        self.__lavrage['lavrage'] /= self.__lavrage['lavrage'].sum()

    
    def __str__(self) -> str:
        self.update()
        return str(self.__lavrage)
    
    
    
    def print(self) -> None:
        self.update()
        print('risk: ', self.risk)
        print('return: ', self._return)
        print('fitness: ', self.fitness)
        
        
    

In [4]:
class Evolution:
    population: dict[Chromosome: float]
    firstPopulation = dict[Chromosome: float]
    n: int
    
    def __init__(self, n: int):
        self.n = n
        self.population = {} 
        self.firstPopulation = {}
        
    def generateRandomChromosome(self, n: int)-> Chromosome:
        lavrage = numpy.random.rand(n)
        lavrage = lavrage/lavrage.sum()
        c = Chromosome(lavrage)
        return c
    
    def firstGeneration(self, path: str) -> None:
        Chromosome.df = pd.read_csv(sample.csv)
        
        for _ in range(self.n):
            c = self.generateRandomChromosome(len(Chromosome.df))  
            self.firstPopulation[c] = c.fitness
        self.firstPopulation = dict(sorted(self.firstPopulation.items(), key=lambda item: -item[1]))
    
    def crossover(self, parent1: numpy.ndarray, parent2: numpy.ndarray, pc: float) -> tuple[numpy.ndarray, numpy.ndarray]:
        if pc > random.uniform(0, 1):
            for i in range(len(parent1)):
                if 0.5 > random.uniform(0, 1): parent1[i] += parent2[i]
                else: parent2[i] += parent1[i]
            parent1, parent2 = parent1/parent1.sum(), parent2/parent2.sum()

        return parent1, parent2
    
    def createNextGeneration(self, pc: float, pm: float) -> None:
        newPopulation = {}
        parents = self.matingPool()
        for i in range(0, len(parents), 2):
            parent1, parent2 = parents[i], parents[i+1]
            offspring1Lvrage, offspring2Lvrage = self.crossover(parent1.lavrage, parent2.lavrage, pc)
            offspring1 = Chromosome(offspring1Lvrage)
            offspring2 = Chromosome(offspring2Lvrage)
            self.mutation(pm, offspring1)
            self.mutation(pm, offspring2)
            newPopulation[offspring1] = offspring1.fitness
            newPopulation[offspring2] = offspring2.fitness
        combinedPopulation = dict(list(self.population.items())[:ELITE] + list(newPopulation.items()))
        sortedPopulation = dict(sorted(combinedPopulation.items(), key=lambda item: -item[1]))
        self.population = dict(list(sortedPopulation.items())[:self.n])

    def mutation(self, pm: float, chromosome: Chromosome) -> None:
         if random.uniform(0, 1) < pm:
            chromosome.mutate()
        

    def matingPool(self) -> list[Chromosome]:
        rank  = [i for i in range(self.n, 0, -1)]
        selected  = []
        keys = list(self.population.keys())
        for i in range(self.n):
            for _ in range(rank[i]):
                selected.append(keys[i])                
        random.shuffle(selected)
        return selected[:self.n]
    
                
        
    def evolution(self, path: str, pc: float, pm: float) -> None:
        done = 0
        while True:
            tst.firstGeneration(path)
            self.population = deepcopy(self.firstPopulation)
            for _ in range(50):
                self.createNextGeneration(pc, pm)
                
                if (list(self.population.keys())[0].risk < 60 and list(self.population.keys())[0]._return > 1000):
                    print(list(self.population.keys())[0])
                    list(self.population.keys())[0].print()
                    done  = 1
                    break
            if done: break

        

    

TypeError: 'type' object is not subscriptable